In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 15 12:21:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
!pip install cellpose  #Put in such lines comment if you're running on notebook!! easier for me later on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 171 kB 28.0 MB/s 
     |████████████████████████████████| 4.9 MB 53.7 MB/s 
     |████████████████████████████████| 35.1 MB 335 kB/s 


In [4]:
!pip install scikit-image==0.19.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.0 MB 32.7 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3


In [5]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [6]:
# Import modules
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
from cellpose import models
from cellpose.io import imread
from cellpose import plot
from skimage.util import img_as_ubyte
import pandas as pd
from skimage.measure import label, regionprops
from skimage.measure import label, regionprops
from skimage import measure

/usr/local/lib/python3.8/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [7]:
import scipy
from scipy import ndimage as ndi
import cv2
from skimage.morphology import square, dilation
from sklearn.cluster import KMeans, DBSCAN
# https://stackoverflow.com/questions/19970764/making-feature-vector-from-gabor-filters-for-classification
# https://stackoverflow.com/questions/20608458/gabor-feature-extraction

# Image segmentation

In [8]:
nb_images = 12

alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

def end_of_file_name(letter,number_tf,number_img,Type):
  """
  letter : 'A' to 'H'
  number : 1 to 12
  type = 'TexasRed' or 'YFP'
  """
  return(letter+' - '+str(number_tf)+'(fld '+str(number_img)+' wv '+Type+' - '+Type+').tif')

Image_Directory = "/content/gdrive/MyDrive/ML_2_transcripted_cherry/TF1/"

***
# Feature extraction

In [11]:
def gabor_feature_extraction(kernels,patch_data):
    # Apply Gabor filter: We receive per kernel a response. We stack all the response to one matrix---------------------------------------------------------
    image = patch_data
    g_response = []

    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        g_response.append(filtered)
    n_clusters = len(g_response)
    g_response = np.vstack(g_response)

    # Cluster Gabor response with k-means. Find which part of the pixel belongs to which Gabor response ---------------------------------------------------------
    random_state = 1
    random.seed(random_state)
    kmean = KMeans(n_clusters, random_state=random_state).fit(g_response) #

    # Texton map: Assign each pixel to the identity of the closest cluster center---------------------------------------------------------
    histogram = np.histogram(kmean.labels_, bins = len(np.unique(kmean.labels_)))
    # Normalize
    gabor_feature = histogram[0] / histogram[0].sum()
    return gabor_feature

In [12]:
def feature_binary_pattern(patch_data):
    # Apply lbp
    METHOD = 'uniform' #  'default'
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(patch_data, n_points, radius, METHOD)
    
    # Create histogram
    count = len(np.unique(lbp))
    H, bins = np.histogram(lbp, count)

    # Drop background values
    w = width = radius - 1
    index_background = n_points - w
    H = H[0: index_background]
    bins =  bins[0: index_background]

    # plt.bar(bins,H,width=1)
    
    # Normalize
    lbp_feature = H / H.sum()
    return lbp_feature

In [13]:
from skimage.feature import local_binary_pattern
from skimage.color import label2rgb
from skimage.transform import rotate

In [14]:
# Initialize list of features 
columnname = ["" for i in range(1)]
patch_size = [0 for i in range(1)]
mean_intensity = [0 for i in range(1)]
sum_intensity = [0 for i in range(1)]
median_intensity = [0 for i in range(1)]
variance_intensity = [0 for i in range(1)]
imgs = [-1 for i in range(1)]

In [15]:
# prepare Gabor filter bank kernels---------------------------------------------------------
kernels = []
sigmas = [1,3]
tilts = [0,0.25,0.5,0.75] # perpendicular, diagonal, horizontal, other diagonal 
Wavelengths = [0.25,0.75] 
gamma =  0.5 # Spatial aspect ratio
psi = 0 # Phase offset.

for tilt in tilts:
    for sigma in sigmas:
        for lambd in Wavelengths:
            kernel = cv2.getGaborKernel((10,10), sigma, tilt, lambd, gamma, psi, ktype = cv2.CV_32F)
            kernels.append(kernel)

In [16]:
from skimage.feature import local_binary_pattern

In [17]:
#Automatization ! 

Image_Directory = "/content/gdrive/MyDrive/ML_2_transcripted_cherry/TF1/"
cherry_condition = "TexasRed"
ypet_condition = "YFP"

channels = [0,0] # Corresponds to greyscale image
diameter = None # Model was trained on diameters of 17 pixels. None-> automated estimation of the diameter. Can be changed by hand later
flow_threshold = 0.4 # maximum allowed error of the flows for each mask, default = 0.4
gpu = True
model_type='nuclei'

# Set model
model = models.Cellpose(gpu, model_type)

let = 'C' #letter of the TF file to be analyzed, could also be automatized

for k in np.arange(0,12):
  Cherry_file = [Image_Directory+end_of_file_name(let,k+1,i+1,cherry_condition) for i in range(12)]
  Ypet_file = [Image_Directory+end_of_file_name(let,k+1,i+1,ypet_condition) for i in range(12)]

  # Initialize lists containing cherry, ypet images
  cherry_imgs = np.ndarray(nb_images,dtype = np.ndarray)
  ypet_imgs = np.ndarray(nb_images,dtype = np.ndarray)

  # Read image (We donnot add it in for loop above to enable reading subset of image)
  ii = 0
  jj = 0
  for i in range(12): #For loop to avoid trying to read images that does not exist
      #print(Cherry_file[ii])
      if (os.path.isfile(Cherry_file[ii])):
        cherry_imgs[ii] = imread(Cherry_file[ii])
        ii = ii+1
      if (os.path.isfile(Ypet_file[jj])):
        ypet_imgs[jj] = imread(Ypet_file[jj])
        jj=jj+1
      
  cherry_imgs = cherry_imgs.tolist()
  ypet_imgs = ypet_imgs.tolist()

  cherry_imgs = cherry_imgs[0:ii]
  ypet_imgs = ypet_imgs[0:jj]
  #print('ii is ',ii)
  #print(jj)
  # Nuclei segmentation of all images within cherry_imgs list
  masks, flows, styles, diams = model.eval(cherry_imgs, diameter=diameter, channels=channels, flow_threshold=flow_threshold, do_3D=False)

  imgs = [-1 for i in range(1)]
  range_s = 1 #Just to initialize the Ypet_glob
  columnname = ["" for i in range(1)]
  blur = [False for i in range(1)]
  patch_size = [0 for i in range(1)]
  mean_intensity = [0 for i in range(1)]
  sum_intensity = [0 for i in range(1)]
  median_intensity = [0 for i in range(1)]
  std_intensity = [0 for i in range(1)]
  var_intensity = [0 for i in range(1)]
  skew_intesity = [0 for i in range(1)]
  kurt_intesity = [0 for i in range(1)]
  iqr_intesity = [0 for i in range(1)]
  entropy_intesity = [0 for i in range(1)]
  gabor_features = [0 for i in range(1)]
  lbp_features = [0 for i in range(1)]
  blur_lapl = [False for i in range(1)]
  blur_ski = [False for i in range(1)]
  TF_name = ['None']
  ypet_intensity_glob = pd.DataFrame(index=columnname)
  ypet_intensity_glob['img'] = imgs
  ypet_intensity_glob['fld'] = ypet_intensity_glob['img']+1 #To remember the flow from which (img+1)
  ypet_intensity_glob['TF_name'] = TF_name
  ypet_intensity_glob['blur_lapl'] = blur_lapl
  ypet_intensity_glob['blur_ski'] = blur_ski
  ypet_intensity_glob['patch_size'] = patch_size
  ypet_intensity_glob['mean_intensity'] = mean_intensity
  ypet_intensity_glob['sum_intensity'] = sum_intensity
  ypet_intensity_glob['median_intensity'] = median_intensity
  ypet_intensity_glob['standard_deviation'] = std_intensity 
  ypet_intensity_glob['variance'] = var_intensity
  ypet_intensity_glob['skewness'] = skew_intesity
  ypet_intensity_glob['kurtosis'] = kurt_intesity
  ypet_intensity_glob['interquartile_range ']= iqr_intesity
  ypet_intensity_glob['entropy'] = entropy_intesity

      
  for j in range(min(ii,jj)):
    # Initialize features: Faster then appending values to list with append
    mask =  masks[j]  
    TF_name = let+' - '+str(k+1)  #TF_name_tot[j]
    start_nucleus = 1 # we have to at least start from 1, otherwise we take whole mask
    end_nucleus =  len(np.unique(mask)) # len(np.unique(mask))  #IMAGE SPECIFIC: cases where cell are big enough (sell last cell of the notebook)
    range_s = range(start_nucleus, end_nucleus)
    imgs = [j for i in range_s]
    ypet = ypet_imgs[j]
    columnname = ["" for i in range_s]
    blur = [False for i in range_s]
    patch_size = [0 for i in range_s]
    mean_intensity = [0 for i in range_s]
    sum_intensity = [0 for i in range_s]
    median_intensity = [0 for i in range_s]
    std_intensity = [0 for i in range_s]
    var_intensity = [0 for i in range_s]
    skew_intesity = [0 for i in range_s]
    kurt_intesity = [0 for i in range_s]
    iqr_intesity = [0 for i in range_s]
    entropy_intesity = [0 for i in range_s]
    gabor_features = [0 for i in range_s]
    lbp_features = [0 for i in range_s]
    blur_lapl = [False for i in range_s]
    blur_ski = [False for i in range_s]
                  
    #calculate different properties of the mask
    props = regionprops(mask)
    
    #initialize arrays to store information from the mask
    perimeter = [0 for i in range_s]
    feret_diameter_max = [0 for i in range_s]
    area_test = [0 for i in range_s]
    axis_major = [0 for i in range_s]
    axis_minor = [0 for i in range_s]
    solidity = [0 for i in range_s]

    '''
    # Iterate to explore and find good threshold in exploration phase
    patch_data_list = []
    variance_data_list = []'''
    # Threshold for blur detection
    var_threshold = 10000000
    blur_threshold = 0.7
    
    # Iterate through all nucleus: Take submatrix, Calculate patch size and corresponding ypet signal (mean) and store it into ypet_intensity
    iter_i = 0 # Ensures that no 0 are saved where patch is too small/big
    
    for i in range_s:# # Iterate through all nucleus: in range(start_nucleus, end_nucleus)
        cur = ypet[mask==i]
        patch_size_ = np.count_nonzero(~np.isnan(cur))
        
        # Extract subimage ----------------------------------------------------------------
        # Extract subimage of nucleus (Not just its values, which is done above)
        boolean = np.where(mask==i, 1,False) # Boolean matrix where patch = 1, else 0
        patch_values = ypet*boolean
        # Extract submatrix with a buffer of 3 pixels to calculate the blurrness
        variance_boolean = dilation(boolean, square(7))
        variance_values = ypet * variance_boolean
        
        # Drop pixels without nucleus information ----------------------------------------------------------------
        # Initialize conditions on when a column/ row should be dropped
        drop_nucleus = False
        drop_column = []
        drop_column_variance = []
        drop_row = []
        drop_row_variance = []
        
        
        # Iterate through all columns to find nucleus that is on the border and the columns that only contain mask
        for column in range(boolean.shape[1]):
            
            # drop nucleus if it's on the border 
            if (((column == 0) | (column == boolean.shape[1]-1)) & (np.any(boolean[:,column])== True)):
                drop_nucleus = True
                break 
                
            # drop if column has no information about nucleus: first check for broader variance_submatrix (Matrix with a buffer around submatrix to calculate variance). Then smaller submatrix 
            if np.any(variance_boolean[:,column])== False:
                drop_column_variance.append(column) 
                drop_column.append(column) 
            elif np.any(boolean[:,column])== False: 
                drop_column.append(column) 
        
        # Iterate through all columns to find nucleus that is on the border and the columns that only contain mask
        for row in range(boolean.shape[0]):
            
            # drop nucleus if it's on the border 
            if (((row == 0) | (row == boolean.shape[0]-1)) & (np.any(boolean[row,:])== True)):
                drop_nucleus = True
                break
                
            # drop if row has no information about nucleus:  first check for broader variance_submatrix  
            if np.any(variance_boolean[row,:])== False:
                drop_row.append(row)
                drop_row_variance.append(row)
            elif np.any(boolean[row,:])== False: 
                drop_row.append(row)
        
        # Drop pixels without nucleus information
        patch_data = np.delete(patch_values,drop_column,1)  
        patch_data = np.delete(patch_data,drop_row,0) 
        variance_data = np.delete(ypet,drop_column_variance,1)  
        variance_data = np.delete(variance_data,drop_row_variance,0) 
        
        '''
        # Exploration phase
        patch_data_list.append(patch_data)
        variance_data_list.append(variance_data)'''
        
        # Calculate blurness based on laplacian----------------------------------------------------------------
        var_lapl = cv2.Laplacian(variance_data, cv2.CV_64F).var()
        if var_lapl >= var_threshold: # nucleus is blurry
            blured_nucleus = True
        else: 
            blured_nucleus = False
        
        # Calculate blurness based on skimage----------------------------------------------------------------
        blur_ski_nucleus = measure.blur_effect(variance_data)
        if blur_ski_nucleus >= blur_threshold:
            blur_ski_temp = True
        else:
            blur_ski_temp = False
        
        # Feature calculation ----------------------------------------------------------------
        
        if ((patch_size_ > 10) & (patch_size_ < 50000) & (drop_nucleus== False)): 
            # extract intensity feature
            columnname[iter_i] = "patch_" + str(iter_i)    #We should not forgot that there is a -1 here. (when it is here)               
            patch_size[iter_i] = patch_size_
            blur_lapl[iter_i] = blured_nucleus
            blur_ski[iter_i] = blur_ski_temp
            sum_intensity[iter_i] = np.sum(np.sum(cur))
            mean_intensity[iter_i] = np.nanmean(cur)
            median_intensity[iter_i] = np.nanmedian(cur)
            
            std_intensity[iter_i] = np.nanstd(cur)
            var_intensity[iter_i] = np.nanvar(cur)
            skew_intesity[iter_i] = scipy.stats.skew(cur)
            kurt_intesity[iter_i] = scipy.stats.kurtosis(cur)
            iqr_intesity[iter_i] = scipy.stats.iqr(cur)
            entropy_intesity[iter_i] = scipy.stats.entropy(cur)
            
            gabor_features[iter_i] = gabor_feature_extraction(kernels,patch_data) # List of two arrays, first nb of counts in histogram, second: bin number
            temp_lbp = feature_binary_pattern(patch_data)
            lbp_features[iter_i]  = temp_lbp
            lbp_count = len(temp_lbp)
            
            perimeter[iter_i] = props[i-1].perimeter
            feret_diameter_max[iter_i] = props[i-1].feret_diameter_max
            area_test[iter_i] = props[i-1].area
            axis_major[iter_i] = props[i-1].major_axis_length
            axis_minor[iter_i] = props[i-1].minor_axis_length
            solidity[iter_i] = props[i-1].solidity
            
            iter_i +=1
    
    # Drop empty rows: some patches values were not calculated because their size is too small/big 
    if iter_i != range_s: # Some patches were not considered -> we have to drop those rows
        mean_intensity = mean_intensity[:iter_i]
        columnname = columnname[:iter_i]
        blur_lapl = blur_lapl[:iter_i]
        blur_ski = blur_ski[:iter_i]
        patch_size = patch_size[:iter_i]
        sum_intensity = sum_intensity[:iter_i]
        median_intensity = median_intensity[:iter_i]
        imgs = imgs[:iter_i]
        std_intensity = std_intensity[:iter_i] 
        var_intensity = var_intensity[:iter_i] 
        skew_intesity = skew_intesity[:iter_i] 
        kurt_intesity = kurt_intesity[:iter_i] 
        iqr_intesity = iqr_intesity[:iter_i]
        entropy_intesity = entropy_intesity[:iter_i] 
        gabor_features = gabor_features[:iter_i] 
        lbp_features = lbp_features[:iter_i] 
        
        perimeter = perimeter[:iter_i]
        feret_diameter_max = feret_diameter_max[:iter_i]
        area_test = area_test[:iter_i]
        axis_major = axis_major[:iter_i]
        axis_minor = axis_minor[:iter_i]
        solidity = solidity[:iter_i]
        
    ypet_intensity = pd.DataFrame(index=columnname)
    ypet_intensity['img'] = imgs
    ypet_intensity['fld'] = ypet_intensity['img']+1 #To remember the flow from which (img+1)
    ypet_intensity['TF_name'] = TF_name
    ypet_intensity['blur_lapl'] = blur_lapl
    ypet_intensity['blur_ski'] = blur_ski
    ypet_intensity['patch_size'] = patch_size
    ypet_intensity['mean_intensity'] = mean_intensity
    ypet_intensity['sum_intensity'] = sum_intensity
    ypet_intensity['median_intensity'] = median_intensity
    
    ypet_intensity['standard_deviation'] = std_intensity 
    ypet_intensity['variance'] = var_intensity
    ypet_intensity['skewness'] = skew_intesity
    ypet_intensity['kurtosis'] = kurt_intesity
    ypet_intensity['interquartile_range ']= iqr_intesity
    ypet_intensity['entropy'] = entropy_intesity
    ypet_intensity['Perimeter'] = perimeter 
    ypet_intensity['area_Test'] = area_test
    ypet_intensity['axis_major_length'] = axis_major
    ypet_intensity['feret_diameter_max'] = feret_diameter_max
    ypet_intensity['axis_minor_length'] = axis_minor
    ypet_intensity['solidity'] = solidity
    
    if imgs == []:
        print('All nucleis have been droppped. In order to extract features you need nucleus that are not by the border and not blurry.')
    else: 
        gabor_features_tot = np.vstack(gabor_features)
        lbp_features_tot = np.vstack(lbp_features)

        for filter_bank in range (len(kernels)):
            name = 'gabro_'+ str(filter_bank)
            ypet_intensity[name] = gabor_features_tot[:,filter_bank]


        for lbp_features_count in range (lbp_features_tot.shape[1]):
            name = 'lbp_' + str(lbp_features_count)
            ypet_intensity[name] = lbp_features_tot[:,lbp_features_count]
    ypet_intensity_glob = ypet_intensity_glob.append(ypet_intensity)  #pd.concat([ypet_intensity_glob,ypet_intensity])
    print('size of ypet_intisity glob +1 df (=',j+1,')! (usually over 12')
            

  path = '/content/gdrive/MyDrive/ML_2_transcripted_cherry/Features/TF1_'+let+str(k+1)+'.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    ypet_intensity_glob.to_csv(f)

    print('hey, it is saved!')

100%|██████████| 25.3M/25.3M [00:03<00:00, 7.76MB/s]
100%|██████████| 3.54k/3.54k [00:00<00:00, 1.51MB/s]


hey, it is saved!
hey, it is saved!


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


All nucleis have been droppped. In order to extract features you need nucleus that are not by the border and not blurry.
All nucleis have been droppped. In order to extract features you need nucleus that are not by the border and not blurry.
All nucleis have been droppped. In order to extract features you need nucleus that are not by the border and not blurry.
hey, it is saved!


ValueError: ignored

In [ ]:
print('/content/gdrive/MyDrive/ML_2_transcripted_cherry/TF1/A - 1(fld 1 wv TexasRed - TexasRed).tif')
print(Image_Directory+end_of_file_name(let,k+1,1,cherry_condition))


/content/gdrive/MyDrive/ML_2_transcripted_cherry/TF1/A - 1(fld 1 wv TexasRed - TexasRed).tif
/content/drive/MyDrive/ML_2_transcripted_cherry/TF1/A - 1(fld 1 wv TexasRed - TexasRed).tif


### Add column with circularity

Formula for circularity: 4*pi*area/(perimeter^2)
Zero would mean that it is a circle. Everything else is between zero and one.

In [ ]:
ypet_intensity['Circularity'] = (4*np.pi*ypet_intensity['patch_size'])/(ypet_intensity['Perimeter']**2)

In [ ]:
ypet_intensity.head()

,img,TF_name,blur_lapl,blur_ski,patch_size,mean_intensity,sum_intensity,median_intensity,standard_deviation,variance,...,gabro_15,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,lbp_6,lbp_7,Circularity
patch_0,0,A - 1,False,False,983,4423.655137,4348453,2839.0,3111.904859,9.683952e+06,...,0.004934,0.136302,0.145749,0.110661,0.130904,0.170040,0.120108,0.072874,0.113360,0.885280
patch_1,0,A - 1,False,False,815,6564.204908,5349827,7561.0,2752.081685,7.573954e+06,...,0.006048,0.086257,0.096491,0.102339,0.163743,0.255848,0.128655,0.076023,0.090643,0.872776
patch_2,0,A - 1,False,False,499,9664.827655,4822749,9789.0,1686.611866,2.844660e+06,...,0.005859,0.089862,0.099078,0.094470,0.214286,0.188940,0.172811,0.076037,0.064516,0.879739
